# 2-way Single Intersection

In [1]:
import csv
import os

import pandas as pd
import pyautogui

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack
from sumo_rl import SumoEnvironment

from evaluate import evaluate
from helper_functions import sumo_vec_env

In [2]:
from stable_baselines3.common.utils import set_random_seed

TEST_NUM = 14
SEED = 0
set_random_seed(SEED)

## Create Environment and Instantiate Agent

In [3]:
from reward_functions import combined_reward
from envs import CountAllRewardsEnv

csv_dir = os.path.join("outputs","2way_single_intersection",f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

env_params = {
    "net_file": os.path.join("nets","2way_single_intersection","single-intersection.net.xml"),
    "route_file": os.path.join("nets","2way_single_intersection","single-intersection-vhvh.rou.xml"),
    "num_seconds": 3600,
    "single_agent": True,
    "reward_fn": combined_reward,
    "sumo_seed": 23423,  # default SUMO seed
}
congestion_reward = combined_reward.__defaults__[0].__name__
alpha = combined_reward.__defaults__[1]  # congestion component coefficient

STACK_NUM = 3

In [4]:
# Create training and evaluation envs
eval_env = sumo_vec_env(seed=SEED, env_kwargs=env_params)
eval_env = VecFrameStack(eval_env, STACK_NUM)

train_env = sumo_vec_env(num_envs=8, seed=SEED, env_kwargs=env_params)
train_env = VecFrameStack(train_env, STACK_NUM)

In [5]:
from hyperparams import custom

model = PPO(
    "MlpPolicy",
    train_env,
    tensorboard_log=os.path.join("logs","2way_single_intersection"),
    seed=SEED,
    verbose=1,
    **custom,
)

Using cuda device


## Evaluate Random Agent

In [6]:
# Evaluate untrained random agent
csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs","2way_single_intersection","eval_untrained")

reward_untrained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

print(reward_untrained)

-313445.8110771179


### Record Results

In [7]:
untrained_csv = pd.read_csv(os.path.join(csv_dir, "untrained.csv"))
arrived_untrained = sum(untrained_csv.arrived_num)
pm_untrained = sum(untrained_csv.tyre_pm)
final_wait_untrained = untrained_csv.waiting_time.iat[-1]
# pressure_untrained = -sum(untrained_csv.pressure)

data = [TEST_NUM, congestion_reward, alpha, 0,
        reward_untrained, arrived_untrained, pm_untrained, final_wait_untrained]

with open("2way_single_intersection_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

## Train Agent

In [8]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)
eval_callback = EvalCallback(eval_env, callback_after_eval=stop_train_callback, eval_freq=25000,
                             best_model_save_path=os.path.join("saved_models",f"2way_single_intersection_{TEST_NUM}"))

TRAIN_STEPS = 1e6
model.learn(total_timesteps=TRAIN_STEPS, callback=eval_callback)

Logging to logs\2way_single_intersection\PPO_15
-----------------------------
| time/              |      |
|    fps             | 32   |
|    iterations      | 1    |
|    time_elapsed    | 62   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 29           |
|    iterations           | 2            |
|    time_elapsed         | 138          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 7.232942e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.39        |
|    explained_variance   | -0.000226    |
|    learning_rate        | 0.00025      |
|    loss                 | 1.81e+06     |
|    n_updates            | 4            |
|    policy_gradient_loss | -0.00011     |
|    value_loss           | 3.51e+06     |
-----------

In [9]:
# Evaluate trained agent
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs","2way_single_intersection",f"PPO_{TEST_NUM}","eval_trained")

reward_trained, _ = evaluate(model, eval_env, csv_path, tb_log_dir, n_eval_episodes=1)

print(reward_trained)

-70211.75290509686


In [10]:
# Move untrained agent's TensorBoard log into numbered folder
import shutil

original = os.path.join("logs","2way_single_intersection","eval_untrained")
target = os.path.join("logs","2way_single_intersection",f"PPO_{TEST_NUM}","eval_untrained")

shutil.move(original, target)

'logs\\2way_single_intersection\\PPO_14\\eval_untrained'

### Record Results

In [48]:
trained_csv = pd.read_csv(os.path.join(csv_dir, "trained.csv"))
arrived_trained = sum(trained_csv.arrived_num)
pm_trained = sum(trained_csv.tyre_pm)
final_wait_trained = trained_csv.waiting_time.iat[-1]
# pressure_trained = -sum(trained_csv.pressure)

data = [TEST_NUM, congestion_reward, alpha, TRAIN_STEPS,
        reward_trained, arrived_trained, pm_trained, final_wait_trained]

with open("2way_single_intersection_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

### Save Model

In [12]:
PPO.save(model,os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

## Render Video

In [42]:
env_params.update({"use_gui": True})
render_env = sumo_vec_env(seed=SEED, env_kwargs=env_params)
render_env = VecFrameStack(render_env, STACK_NUM)

In [49]:
model = PPO.load(os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

obs = render_env.reset()

folder_path = os.path.join("renders","2way_single_intersection",f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

max_time = render_env.envs[0].unwrapped.sim_max_time
delta_time = render_env.envs[0].unwrapped.delta_time
vid_length = int(max_time/delta_time)

for i in range(vid_length):
    actions, _ = model.predict(obs, deterministic=True)
    render_env.step(actions)

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

In [50]:
# Clean up
render_env.close()

In [51]:
import subprocess

subprocess.run([
    "ffmpeg", "-y", "-r", "30", "-i",
    f"renders\\2way_single_intersection\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\2way_single_intersection_{TEST_NUM}.mp4"
])

CompletedProcess(args=['ffmpeg', '-y', '-r', '30', '-i', 'renders\\2way_single_intersection\\test_14\\img%d.jpg', 'videos\\2way_single_intersection_14.mp4'], returncode=0)